<a href="https://colab.research.google.com/github/KasperczakMikolaj/example-mikolaj3/blob/main/Notatki_Miko%C5%82aj_Kasperczak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalacja OpenJDK, pobieranie i rozpakowywanie Apache Spark, instalacja findspark i pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0
!pip install -q pyspark==3.2.1

# Ustawienie zmiennych środowiskowych
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

# Inicjalizacja findspark i pyspark
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

# Sprawdzenie, czy istnieje SparkContext, zanim utworzymy nowy
if not SparkContext._active_spark_context:
    sc = SparkContext('local[*]')
else:
    sc = SparkContext.getOrCreate()

# Inicjalizacja SparkSession
spark = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()


In [ ]:
#RDD API. Wyświetl top 5 krajów pod względem liczby łącznych zgonów
 #(total_deaths) w roku 2020. Do każdej krotki dodaj na końcu napis
 #"TOTAL DEATHS IN 2020" UWAGA! W raporcie niektóre kraje są podzielone
 #na regiony (np. USA). W takich przypadkach musisz je zgrupować do poziomu
 # kraju.
death_data = sc.textFile('/content/all_weekly_excess_deaths.csv')
death_data.take(5)

['country;region;region_code;start_date;end_date;days;year;week;population;total_deaths;covid_deaths;expected_deaths;excess_deaths;non_covid_deaths;covid_deaths_per_100k;excess_deaths_per_100k;excess_deaths_pct_change',
 'Australia;Australia;0;01/01/2020;07/01/2020;7;2020;1;25734100;2497 deaths;0;2463.11165730355;33.8883426964494;2497;0;0.131686527589655;0.0137583461131225',
 'Australia;Australia;0;08/01/2020;14/01/2020;7;2020;2;25734100;2510 deaths;0;2458.27832397007;51.721676029927;2510;0;0.200984981133698;0.0210397966436924',
 'Australia;Australia;0;15/01/2020;21/01/2020;7;2020;3;25734100;2501 deaths;0;2436.44499063674;64.5550093632601;2501;0;0.250853961721063;0.0264955743352897',
 'Australia;Australia;0;22/01/2020;28/01/2020;7;2020;4;25734100;2597 deaths;0;2436.61165730341;160.388342696594;2597;0;0.623252193379967;0.065824335287838']

In [ ]:
#(DataFrame API) W jakim tygodniu w każdym z krajów nastąpił
#największy przyrost liczby zgonów (total_deaths) względem liczby z
#poprzedniego tygodnia? (Różnica między total_deaths w danym tygodniu
#a tygodniu poprzedzającym) Wynik posortuj malejąco według różnic
#i wyświetl top 5 krajów z pokazanymi tygodniami i wartościami tak jak w
#przykładowym wyniku.
 (death_data
  .map(lambda x: x.split(';'))
  .filter(lambda x: 'country' not in x)
  .map(lambda x:(x[0],x[6],x[9].split(' ')[0]))
  .filter(lambda x: x[1] == '2020')
  .map(lambda x: (x[0],int(x[2])))
  .reduceByKey(lambda x,y:x+y)
  ).top(5,key=lambda x:x[1])

[('United States', 6964329),
 ('Britain', 2006607),
 ('Italy', 1468112),
 ('France', 1309358),
 ('Mexico', 1065198)]

In [ ]:
# pokazywanie 5 pierwszych
!head -5 (nazwa csv)

In [ ]:
#Wyświetl wartość poszczególnych branż (Industry) w Stanach Zjednoczonych
#pod względem zsumowanego majątku najbogatszych ludzi pochodzących z tych
#branż (pogrupuj branże po Total Net Worth). Dodaj na końcu kolumnę z
# tekstem 'dollars'. Usuń pozycję która zawiera pusty tekst ''.

rich = sc.textFile('500_richest.csv')

rich_no_header = rich.filter(lambda x: x!='Name;Total Net Worth;$ Last Change;$ YTD Change;Country;Industry')

rich_no_header= rich_no_header.map(lambda x: x.split(';'))
rich_no_header.take(1)


#tworzenieKrotek
#zatąpienie znaku $
rich_no_header = rich_no_header.map(lambda x: (x[5], x[1].replace('$', '')))
#Zmina 1.8B na konkretną liczbe
rich_no_header = rich_no_header.map(lambda x: (x[0], x[1].replace('B', '')))
rich_no_header = rich_no_header.map(lambda x: (x[0], float(x[1]) *1000000000))

#Usunięcie pustej wartości
rich_no_header=rich_no_header.filter(lambda x: x[0])

rich_no_header.take(10)

(rich_no_header.map(lambda x: (x[0],int(x[1])))
    .reduceByKey(lambda x,y:x+y)
    .map(lambda x:(x[0],x[1],'dollars')).collect()

   )


In [ ]:
#Rozpatrzmy następujące kraje: Rosja, Szwecja, Niemcy
#(kolumna Country: Russia, Sweden, Germany).
#Wyświetl czwartego najbogatszego człowieka w każdym z tych krajów
#oraz łączną wartość branży wśród najbogatszych ludzi
 #(suma Total Net Worth) w kraju w którym występuje ta osoba w rankingu.
 # Możesz pominąć BLN przy sumowaniu (Industry_Total_In_Country_BLN)
from pyspark.sql.functions import *
import sys
from pyspark.sql.window import Window
from pyspark.sql.functions import lag
from pyspark.sql import functions as f

rich_2=spark.read.format('csv').option("header", True).option("delimiter", ";").load('500_richest.csv')


#Pozostawienie tylko kolumn na których bede pracować
rich_2 =rich_2.select("Country", "Name", 'Industry', 'Total Net Worth')

#Usuwanie zbędnych znaków i rzutowanie na float

rich_2 = rich_2.withColumn("Total Net Worth", f.translate(f.col("Total Net Worth"), "$#,", "").alias("replaced"))

rich_2 = rich_2.withColumn("Total Net Worth", f.regexp_replace("Total Net Worth", "B", "").cast("float"))




rich_2.show()



#grupowanie

df_rich = (rich_2.groupBy("Country",'Industry')
            .agg(round(sum("Total Net Worth"),2)
            .alias("Industry_Total_In_Country_BLN"), first('Name').alias("Name")))
df_rich.show()

In [ ]:
# Sortowanie danych według kolumny "Industry_Total_In_Country_BLN" malejąco
df_rich = df_rich.orderBy("Industry_Total_In_Country_BLN", ascending=False)

# Definicja specyfikacji okna - grupowanie według kraju i sortowanie według "Industry_Total_In_Country_BLN" malejąco
windowSpec = (Window
              .partitionBy(f.col('Country'))
              .orderBy(f.col("Industry_Total_In_Country_BLN").desc())
              )

# Dodanie kolumny "row_number", która przypisuje numer wiersza w obrębie każdej grupy krajów
new_df = df_rich.withColumn('row_number', row_number().over(windowSpec))

# Filtrowanie wierszy, aby pozostawić tylko te, które mają "row_number" równy 4
new_df = new_df.filter(f.col('row_number') == 4)

# Wybieranie kolumn "Country", "Name", "Industry" oraz "Industry_Total_In_Country_BLN"
new_df = new_df.select("Country", "Name", 'Industry', 'Industry_Total_In_Country_BLN')

# Konwersja wynikowego DataFrame do Pandas DataFrame
final_answer = new_df.toPandas()

# Wyświetlanie wynikowego DataFrame
new_df.show()

# Zwrócenie wynikowego Pandas DataFrame
final_answer

In [ ]:
#Załaduj plik pracownicy.csv z poprawnie zdefiniowaną strukturą i
#typami danych (schema).#Dla każdego roku urodzenia występującego w danych
#wylicz średnią stawkę godzinową posortuj od najmłodszych lat do najstarszych

!head -5 (plik)
schema_workers = StructType([StructField('imie', StringType()),
                             StructField('nazwisko', StringType()),
                             StructField('stawka_godzinowa_w dolarach', IntegerType()),
                             StructField('rok_urodzenia', IntegerType())])
df_workers = spark.read.csv("pracownicy.csv", header=True, sep=";", schema=schema_workers).toPandas()
df_workers.head(5)
fragment=df_workers[['stawka_godzinowa_w dolarach', 'rok_urodzenia']]
fragment_agg=fragment.groupby(["rok_urodzenia"]).mean('stawka_godzinowa_w dolarach')
fragment_agg_sort=fragment_agg.sort_values(by='rok_urodzenia', ascending=False)
fragment_agg_sort
fragment_agg_sort.to_excel("wyniki_lata_stawka_sort.xlsx")

#zoptymalizowany kod
df = spark.read.csv("pracownicy.csv", header=True, sep=";", schema=schema_workers)
x=(df.select(f.col("rok _urodzenia"), f.col("stawka_godzinowa_w_dolarach"))
.groupBy(f.col("rok_urodzenia")).mean("stawka_godzinowa_w_dolarach")
.orderBy("rok_urodzenia", ascending=False))

x.toPandas().to_excel("outpus_zad4.xlsx")


In [ ]:
#Jakiego dnia ile było wejść a angielską wikipedię?
#Rozwiąż na dwa sposoby (wykorzystując czyste RDD oraz Dataframes)
#kod 1 (RDD)
(pagecounts
          .map(lambda x: (x[:8], x.split()[1], int(x.split()[3])))  # bierze date i liczbę
          .filter(lambda x: x[1]=="en")
          .map(lambda x: (x[0], x[2]))
          .groupByKey()  # Grupuje po kluczu
          .mapValues(sum)  # sumuje
          .collect())  # wyświetla


In [ ]:
(pagecounts
  .map(lambda x: x.split(' '))
  .filter(lambda x: x[1]=='en')
  .map(lambda x: [x[0][:8], int(x[3])])
  .reduceByKey(lambda x,y: x+y)
).take(10)

In [ ]:
pagecounts.take(10)

In [ ]:
#kod 2 (Dataframes)
schema_wiki = StructType([StructField('date', StringType()),
                             StructField('lang', StringType()),
                             StructField('title', StringType()),
                             StructField('views', IntegerType()),
                             StructField('code', IntegerType())])

In [ ]:
pagecount_df=spark.read.csv('pagecounts', sep=' ', schema=schema_wiki)

In [ ]:
pagecount_df.show(2)

In [ ]:
pagecount_df.printSchema()

In [ ]:
pagecount_df.createOrReplaceTempView("pagecount_df")

In [ ]:
def f_do_wycinania_znakow(x):
  return x[:8]

In [ ]:
udf_do_wycinania_znakow=udf(f_do_wycinania_znakow, StringType())

In [ ]:
(pagecount_df
  .filter(f.col('lang')=="en")
  .select(f.col('date'), f.col('views'))
  .withColumn('date', udf_do_wycinania_roku(f.col('date')))
  .groupBy('date')
  .sum('views')
).show()

In [ ]:
spark.sql(""" SELECT SUBSTR(date, 1, 8) AS DATE, SUM(views) AS SUM_VIEW_COUNT
            FROM pagecount_df
            WHERE lang = 'en'
            GROUP BY 1""").show()

In [ ]:
#Wczytaj wiki_sample.parquet (moodle) Odpowiedz na pytanie którzy użytkownicy
#edytowali najwięcej stron na wikipedii (top 5)?Wykonaj zadanie na dwa sposoby
# (Dataframe SQL oraz Dataframe programmatic API)

In [ ]:
#1
!unzip wiki_sample.parquet
df_wiki = spark.read.parquet("wiki_sample.parquet")

df_wiki.show()
df_wiki.printSchema()

#Data Frame programatic API
 (df_wiki
  .filter(f.col('username')!="")
  .groupBy('username')
  .count()
  .sort('count', ascending=False)
).show(5)

#SQL
df_wiki.createOrReplaceTempView("wiki_table")

top5_users = spark.sql("""
    SELECT username, COUNT(*) AS count
    FROM wiki_table
    WHERE username != ""
    GROUP BY username
    ORDER BY count DESC
    LIMIT 5
""")

top5_users.show()

In [ ]:
#wczytaj plik bonuses.csv stwórz tabelę o kolumnach employee_id,
#year, max_quarter_bonus, min_quarter_bonus, total_bonus,
#total_bonus_previous_year, total_bonus_2_years_ago

bonuses = spark.read.csv("bonuses.csv", header=True, sep=';', inferSchema=True)

bonuses.show()

bonuses_agg = (bonuses
               .groupBy(["employee_id", "year"]) \
          .agg(f.min('bonus').alias("MIN"),
         f.max('bonus').alias("MAX"),
         f.sum('bonus').alias("total_bonus"))) \
         .orderBy("employee_id", "year")

bonuses_agg.show()

# Definiowanie okna do obliczenia bonusów z poprzednich lat
window_quarter = Window.partitionBy("employee_id").orderBy("quarter")

# Dodawanie kolumn z bonusami z poprzednich lat
result_df = bonuses.withColumn(
    "min_quarter_bonus", f.min('bonus').over(window_quarter)
).withColumn(
    "max_quarter_bonus", f.max('bonus').over(window_quarter)
)
result_df.show()

# Definiowanie okna do obliczenia bonusów z poprzednich lat
window_spec = Window.partitionBy("employee_id").orderBy("year")

# Dodawanie kolumn z bonusami z poprzednich lat
result_df2 = result_df.withColumn(
    "total_bonus", f.sum("bonus", 1).over(window_spec))
# ).withColumn(
#     "total_bonus_2_years_ago",
#     lag("total_bonus", 2).over(window_spec)


result_df.show()



Uczenie maszynowe

In [ ]:
#Do tego samego zbioru danych stworz model klasyfikacji oparty na Random
#Forest (https://spark.apache.org/docs/3.1.1/api/python/reference/api/
#pyspark.ml.classification.RandomForestClassifier.html). Postaraj się
#zrobić go w formie "pipeline".

In [ ]:
!unzip train.csv.zip

In [ ]:
data = spark.read.csv("train.csv", header=True, inferSchema=True, sep=",")

In [ ]:
data.show()

In [ ]:
(train, test) = data.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[x for x in data.columns if x not in ['Id', 'Cover_Type']], outputCol='features')

In [ ]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol='Cover_Type', outputCol='label').fit(data)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees=15, maxDepth=30, seed=42, featuresCol='features', labelCol='label')

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler, labelIndexer, rf])
model_pipeline = pipeline.fit(train)

In [ ]:
predictions = model_pipeline.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
print(accuracy)

In [ ]:
from pyspark.ml.feature import IndexToString

In [ ]:
labelIndexer.labels

In [ ]:
labelConverter=IndexToString(inputCol='prediction', outputCol="Cover_Type_Predicted", labels=labelIndexer.labels)

In [ ]:
#Zaimplementuj hyperparameter tuning dla drzew decyzyjnych. Parametry:
#impurity, ["entropy"]; maxDepth, [10, 30]; maxBins, [30, 300, 100];

In [ ]:
(train, test) = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(seed=42)

In [ ]:
paramGrid = (ParamGridBuilder()
  .addGrid(dt.maxDepth, [10, 30])
  .addGrid(dt.maxBins, [30, 300, 100])
  .addGrid(dt.impurity, ['entropy'])
  .build())


In [ ]:
pipeline = Pipeline(stages=[assembler, labelIndexer, dt, labelConverter])

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3) # use 10 folds in practice

In [ ]:
cvModel = crossval.fit(train)

In [ ]:
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
params

In [ ]:
cvModel.avgMetrics

In [ ]:
best_index = cvModel.avgMetrics.index(max(cvModel.avgMetrics))
best_param_map = params[best_index]
best_param_map

In [ ]:
best_model = cvModel.bestModel

In [ ]:
best_model

In [ ]:
predictions_test = best_model.transform(test)

In [ ]:
accuracy_test = evaluator.evaluate(predictions_test)

In [ ]:
print(f"Accuracy test: {accuracy_test}")

In [ ]:
#Regresja liniowa w Sparku - ile będzie rzutów na koniec 2022? Użyj Regresji Logistycznej

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# Prepare the training data
vector_assembler = VectorAssembler(inputCols=['yr'], outputCol='features')
training = vector_assembler.transform(fga_py) \
                          .withColumn('label', col('fg3a_p36m'))

training.show(5)

In [ ]:
from pyspark.ml.regression import LinearRegression

# Train linear regression model
lr = LinearRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(training)

# Prepare the test data for the year 2022
test_data = spark.createDataFrame([(2022,)], ["yr"])
test_features = vector_assembler.transform(test_data)

# Make predictions for 2022
predictions = lr_model.transform(test_features)
predictions.select('yr', 'prediction').show()

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import when

# Binarizacja etykiety na podstawie mediany
training = training.withColumn('label',
    when(col('fg3a_p36m') < 0.5, 0).
    when((col('fg3a_p36m') > 0.5) & (col('fg3a_p36m') <1.5), 1).
    when((col('fg3a_p36m') >= 1.5) & (col('fg3a_p36m') < 2.5), 2).
    when((col('fg3a_p36m') >= 2.5) & (col('fg3a_p36m') < 3.5), 3).
    otherwise(4)
)

# model
log_reg = LogisticRegression(featuresCol='features', labelCol='label', family='multinomial')
log_reg_model = log_reg.fit(training)

test_data = spark.createDataFrame([(2022,)], ["yr"])
test_features = vector_assembler.transform(test_data)

# Step 6: Make Predictions
predictions = log_reg_model.transform(test_features)
predictions.select('yr', 'probability', 'prediction').show()